In [54]:
import numpy as np

Сначала убедимся с помощью fastqc, что файлы получены из Illumina.

Видим, что качество ридов сильно падает к концу, будем считать, что это так.

GC-content так же выглядит адекватно.

In [2]:
! bwa index MG1655-K12.first400K.fasta

[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.08 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.03 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index ./MG1655-K12.first400K.fasta
[main] Real time: 0.118 sec; CPU: 0.128 sec


In [ ]:
! bwa mem MG1655-K12.first400K.fasta ecoli_400K_err_1.fastq.gz ecoli_400K_err_2.fastq.gz > simple.sam

Для анализа качества исправления ошибок будем пользоваться следующим подходом.

Построим для базовых и исправленных выровненных ридов словари ошибок следующего вида:

#### {<номер рида, forward/backward> : {позиция ошибки в референсе: <значение референса, значение рида>}}

Сравнение исправленных ридов с неисправленными будем осуществлять с помощью таких словарей.

P.S. Данный алгоритм подразумевает под удалениями только удаления целых ридов

In [78]:
def get_errors(sam_f, fasta_f):
    # Read reference
    reference = []
    for line in open(fasta_f):
        if line.startswith('>'):
            continue
        reference.append(line.strip())
    reference = list(''.join(reference))
    reference_len = len(reference)
    total_errors = {}
    good_positions_n = 0
    for num, line in enumerate(open(sam_f)):
        if num and (num % 1e6 == 0):
            print(f"Processed {num} SAM-lines")
        if line.startswith('@'):
            continue
        parts = line.strip().split()
        if parts[2] == '*' or parts[5] == '*':
            continue
        start_pos = int(parts[3])
        if start_pos == 0:
            continue

        read_name = parts[0]
        read_type = 'f' if int(parts[8]) > 0 else 'b'
        errors = {}
        read = list(parts[9])
        read_len = len(read)
        end_pos = min(start_pos + read_len - 1, reference_len - 1)
        reference_region = reference[start_pos-1:end_pos]
        for pos, r in enumerate(reference_region):
            if r != read[pos]: # and read[pos] != 'N':
                errors[start_pos + pos] = (r, read[pos])
            else:
                good_positions_n += 1
        if len(errors) > 0:
            total_errors[(read_name, read_type)] = errors
    return total_errors, good_positions_n

def get_reads_names(sam_f):
    names = set()
    for num, line in enumerate(open(sam_f)):
        if num and (num % 1e6 == 0):
            print(f"Processed {num} SAM-lines")
        if line.startswith('@'):
            continue
        parts = line.strip().split()
        if parts[2] == '*' or parts[5] == '*':
            continue
        start_pos = int(parts[3])
        if start_pos == 0:
            continue
        read_name = parts[0]
        read_type = 'f' if int(parts[8]) > 0 else 'b'
        names.add((read_name, read_type))
    return names

def get_deleted_reads(total_reads, sam_f):
    deleted_reads = total_reads.copy()
    for num, line in enumerate(open(sam_f)):
        if num and (num % 1e6 == 0):
            print(f"Processed {num} SAM-lines")
        if line.startswith('@'):
            continue
        parts = line.strip().split()
        if parts[2] == '*' or parts[5] == '*':
            continue
        start_pos = int(parts[3])
        if start_pos == 0:
            continue
        read_name = parts[0]
        read_type = 'f' if int(parts[8]) > 0 else 'b'
        if (read_name, read_type) in total_reads:
            deleted_reads.remove((read_name, read_type))
    return deleted_reads

def deleted_good_positions(errors, deleted_reads, sam_f):
    n = 0
    for num, line in enumerate(open(sam_f)):
        if num and (num % 1e6 == 0):
            print(f"Processed {num} SAM-lines")
        if line.startswith('@'):
            continue
        parts = line.strip().split()
        if parts[2] == '*' or parts[5] == '*':
            continue
        start_pos = int(parts[3])
        if start_pos == 0:
            continue
        read_name = parts[0]
        read_type = 'f' if int(parts[8]) > 0 else 'b'
        if (read_name, read_type) not in deleted_reads:
            continue
        read = list(parts[9])
        read_len = len(read)
        if (read_name, read_type) not in errors:
            n += read_len
            continue
        errors_n = len(errors[(read_name, read_type)])
        n += (read_len - errors_n)
    return n

def get_stat(first_errors, second_errors, first_sam, second_sam, first_good_n, second_good_n):
    print('Get reads names from first file...')
    first_reads = get_reads_names(first_sam)
    print('Completed.')
    print('Get deleted reads...')
    second_deleted_reads = get_deleted_reads(first_reads, second_sam)
    print('Completed.')    
    
    total_reads = len(first_reads)
    deleted_reads = len(second_deleted_reads)
    total_errors = 0
    corrected_errors = 0
    deleted_errors = 0
    
    print('Start comparing errors (1 -> 2)...')
    for (read, read_type), positions in first_errors.items():
        n = len(positions)
        total_errors += n
        if (read, read_type) in second_deleted_reads:
            deleted_errors += n
            continue
        if (read, read_type) not in second_errors:
            corrected_errors += n
            continue
        second_positions = second_errors[(read, read_type)]
        for pos in positions.keys():
            if pos not in second_positions:
                corrected_errors += 1
    print('Completed.')
    print('Start comparing errors (2 -> 1)...')
    new_errors = 0
    for (read, read_type), positions in second_errors.items():
        n = len(positions)
        if (read, read_type) not in first_errors:
            new_errors += n
            continue
        first_positions = first_errors[(read, read_type)]
        for pos in positions.keys():
            if pos not in first_positions:
                new_errors += 1
    print('Completed.')                
    print('Start calculating deleted good bases...')
    deleted_good_n = deleted_good_positions(first_errors, second_deleted_reads, first_sam)
    print('Completed.')
    
    print('----')
    print(f'Total reads: {total_reads}')
    print(f'Deleted reads: {deleted_reads}({np.round(deleted_reads/total_reads, 4)*100}%)')
    print(f'Total errors: {total_errors}')
    print(f'Deleted errors: {deleted_errors}({np.round(deleted_errors/total_errors, 4)*100}%)')
    print(f'Corrected errors: {corrected_errors}({np.round(corrected_errors/total_errors, 4)*100}%)')
    print(f'New errors: {new_errors}({np.round(new_errors/total_errors, 4)*100}%)')
    print(f'Profit: {np.round((1 - (total_errors - deleted_errors - corrected_errors + new_errors)/total_errors), 5)*100}% less errors')
    
    print('----')
    print(f'Undetected error: {np.round((total_errors - deleted_errors - corrected_errors)/total_errors, 4)*100}%')
    print(f'Detected & corrected error: {np.round(corrected_errors/total_errors, 4)*100}%')
    print(f'Detected & removed error: {np.round(deleted_errors/total_errors, 4)*100}%')
    print(f'Falsely corrected error: {np.round(new_errors/first_good_n, 4)*100}%')
    print(f'Correctly unmodified base: {np.round((first_good_n - deleted_good_n - new_errors)/first_good_n, 4)*100}%')
    print(f'Incorrectly removed base: {np.round(deleted_good_n/first_good_n, 4)*100}%')

In [68]:
simple_errors, simple_good_positions_n = get_errors('simple.sam', 'MG1655-K12.first400K.fasta')

Processed 1000000 SAM-lines
Processed 2000000 SAM-lines


## Trimmomatic

Возьмем стандартные настройки из мануала + будем использовать далее только paired-риды.

In [1]:
! java -jar Trimmomatic-0.39/trimmomatic-0.39.jar PE \
ecoli_400K_err_1.fastq.gz ecoli_400K_err_2.fastq.gz \
trimm_1_paired.fq.gz trimm_1_unpaired.fq.gz \
trimm_2_paired.fq.gz trimm_1_unpaired.fq.gz \
LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:36

TrimmomaticPE: Started with arguments:
 ecoli_400K_err_1.fastq.gz ecoli_400K_err_2.fastq.gz trimm_1_paired.fq.gz trimm_1_unpaired.fq.gz trimm_2_paired.fq.gz trimm_1_unpaired.fq.gz LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:36
Multiple cores found: Using 4 threads
Quality encoding detected as phred33
Input Read Pairs: 1381602 Both Surviving: 1309635 (94.79%) Forward Only Surviving: 38225 (2.77%) Reverse Only Surviving: 28561 (2.07%) Dropped: 5181 (0.37%)
TrimmomaticPE: Completed successfully


In [ ]:
! bwa mem MG1655-K12.first400K.fasta trimm_1_paired.fq.gz trimm_2_paired.fq.gz > trimm.sam

In [69]:
trimm_errors, trimm_good_positions_n = get_errors('trimm.sam', 'MG1655-K12.first400K.fasta')

Processed 1000000 SAM-lines
Processed 2000000 SAM-lines


In [79]:
get_stat(simple_errors, trimm_errors, 'simple.sam', 'trimm.sam', simple_good_positions_n, trimm_good_positions_n)

Get reads names from first file...
Processed 1000000 SAM-lines
Processed 2000000 SAM-lines
Completed.
Get deleted reads...
Processed 1000000 SAM-lines
Processed 2000000 SAM-lines
Completed.
Start comparing errors (1 -> 2)...
Completed.
Start comparing errors (2 -> 1)...
Completed.
Start calculating deleted good bases...
Processed 1000000 SAM-lines
Processed 2000000 SAM-lines
Completed.
----
Total reads: 2762617
Deleted reads: 143396(5.19%)
Total errors: 15105095
Deleted errors: 2785389(18.44%)
Corrected errors: 11017705(72.94%)
New errors: 210222(1.39%)
Profit: 89.98899999999999% less errors
----
Undetected error: 8.62%
Detected & corrected error: 72.94%
Detected & removed error: 18.44%
Falsely corrected error: 0.08%
Correctly unmodified base: 95.5%
Incorrectly removed base: 4.42%


В данном случае, так как Trimmomatic не исправляет ошибки, а только удаляет риды целиком или обрезает их, то все исправления на самом деле относятся к удалениям. 

<table>
    <thead>
    <tr>
        <td></td>
        <td style="width: 100px"><b>Error in corrected reads</b></td>
        <td style="width: 100px"><b>Correct base in corrected reads</b></td>
        <td style="width: 100px"><b>Base is absent in corrected reads</b></td>
    </tr>
    </thead>
    <tbody>
    <tr>
        <td style="width: 100px"><b>Error in raw data</b></td>
        <td style="width: 100px">8.62% Undetected error (FN)</td>
        <td style="width: 100px">0% Detected & corrected error (TP)</td>
        <td style="width: 100px">91.38% Detected and removed error (TP)</td>
    </tr>
    <tr>
        <td style="width: 100px"><b>Correct base in raw data</b></td>
        <td style="width: 100px">0% Falsely corrected error (FP)</td>
        <td style="width: 100px">95.5% Correctly unmodified base (TN)</td>
        <td style="width: 100px">4.5% Incorrectly removed base (FP)</td>
    </tr>
    </tbody>
</table>

## Spades

In [ ]:
./SPAdes-3.14.1-Darwin/bin/spades.py -1 ecoli_400K_err_1.fastq.gz -2 ecoli_400K_err_2.fastq.gz -o spades_res --only-error-correction --threads 8

In [ ]:
! bwa mem MG1655-K12.first400K.fasta spades_res/corrected/ecoli_400K_err_1.fastq.00.0_0.cor.fastq.gz \
spades_res/corrected/ecoli_400K_err_2.fastq.00.0_0.cor.fastq.gz > spades.sam

In [82]:
spades_errors, spades_good_positions_n = get_errors('spades.sam', 'MG1655-K12.first400K.fasta')

Processed 1000000 SAM-lines
Processed 2000000 SAM-lines


In [83]:
get_stat(simple_errors, spades_errors, 'simple.sam', 'spades.sam',
         simple_good_positions_n, spades_good_positions_n)

Get reads names from first file...
Processed 1000000 SAM-lines
Processed 2000000 SAM-lines
Completed.
Get deleted reads...
Processed 1000000 SAM-lines
Processed 2000000 SAM-lines
Completed.
Start comparing errors (1 -> 2)...
Completed.
Start comparing errors (2 -> 1)...
Completed.
Start calculating deleted good bases...
Processed 1000000 SAM-lines
Processed 2000000 SAM-lines
Completed.
----
Total reads: 2762617
Deleted reads: 46681(1.69%)
Total errors: 15105095
Deleted errors: 1079024(7.140000000000001%)
Corrected errors: 4190820(27.74%)
New errors: 37651757(249.27%)
Profit: -214.377% less errors
----
Undetected error: 65.11%
Detected & corrected error: 27.74%
Detected & removed error: 7.140000000000001%
Falsely corrected error: 14.42%
Correctly unmodified base: 84.21%
Incorrectly removed base: 1.37%


Здесь видим, что Spades исправил много верных оснований. Возможно, его необходимо аккуратнее тюнить.

<table>
    <thead>
    <tr>
        <td></td>
        <td style="width: 100px"><b>Error in corrected reads</b></td>
        <td style="width: 100px"><b>Correct base in corrected reads</b></td>
        <td style="width: 100px"><b>Base is absent in corrected reads</b></td>
    </tr>
    </thead>
    <tbody>
    <tr>
        <td style="width: 100px"><b>Error in raw data</b></td>
        <td style="width: 100px">65.11% Undetected error (FN)</td>
        <td style="width: 100px">27.74% Detected & corrected error (TP)</td>
        <td style="width: 100px">7.14% Detected and removed error (TP)</td>
    </tr>
    <tr>
        <td style="width: 100px"><b>Correct base in raw data</b></td>
        <td style="width: 100px">14.42% Falsely corrected error (FP)</td>
        <td style="width: 100px">84.21% Correctly unmodified base (TN)</td>
        <td style="width: 100px">1.37% Incorrectly removed base (FP)</td>
    </tr>
    </tbody>
</table>